# Initialization

In [1]:
import logging

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [2]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
%config InlineBackend.figure_format = 'retina'

# Загрузка данных

In [3]:
items = pd.read_parquet("items.par")
events = pd.read_parquet("events.par")

# Разбиение с учётом хронологии

Рекомендательные системы на практике работают с учётом хронологии. Поэтому поток событий для тренировки и валидации полезно делить на то, что уже случилось, и что ещё случится. Это позволяет проводить валидацию на тех же пользователях, на которых тренировались, но на их событиях в будущем.

# === Знакомство: "холодный" старт

In [4]:
# зададим точку разбиения
train_test_global_time_split_date = pd.to_datetime("2017-08-01").date()

train_test_global_time_split_idx = events["started_at"] < train_test_global_time_split_date
events_train = events[train_test_global_time_split_idx]
events_test = events[~train_test_global_time_split_idx]

# количество пользователей в train и test
users_train = events_train["user_id"].drop_duplicates()
users_test = events_test["user_id"].drop_duplicates()
# количество пользователей, которые есть и в train, и в test
common_users = pd.Series(list(set(users_train).intersection(set(users_test))))
print(len(users_train), len(users_test), len(common_users))

# Идентифицируйте холодных пользователей и оцените их количество.
cold_users = users_test[~users_test.isin(users_train)]

print(len(cold_users))

428220 123223 120858
2365


In [5]:
# получить топ-100 наиболее популярных книг

from sklearn.preprocessing import MinMaxScaler

top_pop_start_date = pd.to_datetime("2015-01-01").date()

item_popularity = events_train \
    .query("started_at >= @top_pop_start_date") \
    .groupby(["item_id"]).agg(users=("user_id", "nunique"), avg_rating=("rating", "mean")).reset_index()

# нормализация пользователей и среднего рейтинга, требуется для их приведения к одному масштабу
scaler = MinMaxScaler()
item_popularity[["users_norm", "avg_rating_norm"]] = scaler.fit_transform(
    item_popularity[["users", "avg_rating"]]
)

# вычисляем popularity_score, как скор популярности со штрафом за низкий рейтинг
item_popularity["popularity_score"] = (
    item_popularity["users_norm"] * item_popularity["avg_rating_norm"]
)

# сортируем по убыванию popularity_score
item_popularity = item_popularity.sort_values(by="popularity_score", ascending=False)

# выбираем первые 100 айтемов со средней оценкой avg_rating не меньше 4
top_k_pop_items = item_popularity.query("avg_rating >= 4").head(100)

top_k_pop_items

,item_id,users,avg_rating,users_norm,avg_rating_norm,popularity_score
32387,18007564,20207,4.321275,0.496596,0.830319,0.412333
32623,18143977,19462,4.290669,0.478287,0.822667,0.393471
2,3,15139,4.706057,0.372042,0.926514,0.344702
30695,16096824,16770,4.301014,0.412126,0.825253,0.340108
1916,15881,13043,4.632447,0.320529,0.908112,0.291076
...,...,...,...,...,...,...
24837,8490112,4792,4.080968,0.117747,0.770242,0.090694
33611,18966819,4361,4.374914,0.107154,0.843729,0.090409
378,3636,4667,4.098564,0.114675,0.774641,0.088832
32835,18293427,4674,4.092640,0.114847,0.773160,0.088795


# === Знакомство: первые персональные рекомендации

In [6]:
# добавляем информацию о книгах
top_k_pop_items = top_k_pop_items.merge(
    items.set_index("item_id")[["author", "title", "genre_and_votes", "publication_year"]], on="item_id")

with pd.option_context('display.max_rows', 100):
    display(top_k_pop_items[["item_id", "author", "title", "publication_year", "users", "avg_rating", "popularity_score", "genre_and_votes"]])

,item_id,author,title,publication_year,users,avg_rating,popularity_score,genre_and_votes
0,18007564,Andy Weir,The Martian,2014,20207,4.321275,0.412333,"{'Science Fiction': 11966, 'Fiction': 8430}"
1,18143977,Anthony Doerr,All the Light We Cannot See,2014,19462,4.290669,0.393471,"{'Historical-Historical Fiction': 13679, 'Fict..."
2,3,"J.K. Rowling, Mary GrandPré",Harry Potter and the Sorcerer's Stone (Harry P...,1997,15139,4.706057,0.344702,"{'Fantasy': 59818, 'Fiction': 17918, 'Young Ad..."
3,16096824,Sarah J. Maas,A Court of Thorns and Roses (A Court of Thorns...,2015,16770,4.301014,0.340108,"{'Fantasy': 14326, 'Young Adult': 4662, 'Roman..."
4,15881,"J.K. Rowling, Mary GrandPré",Harry Potter and the Chamber of Secrets (Harry...,1999,13043,4.632447,0.291076,"{'Fantasy': 50130, 'Young Adult': 15202, 'Fict..."
5,38447,Margaret Atwood,The Handmaid's Tale,1998,14611,4.232770,0.290194,"{'Fiction': 15424, 'Classics': 9937, 'Science ..."
6,11235712,Marissa Meyer,"Cinder (The Lunar Chronicles, #1)",2012,14348,4.179189,0.280247,"{'Young Adult': 10539, 'Fantasy': 9237, 'Scien..."
7,17927395,Sarah J. Maas,A Court of Mist and Fury (A Court of Thorns an...,2016,12177,4.730640,0.279094,"{'Fantasy': 10186, 'Romance': 3346, 'Young Adu..."
8,5,"J.K. Rowling, Mary GrandPré",Harry Potter and the Prisoner of Azkaban (Harr...,2004,11890,4.770143,0.275401,"{'Fantasy': 49784, 'Young Adult': 15393, 'Fict..."
9,13206900,Marissa Meyer,"Winter (The Lunar Chronicles, #4)",2015,12291,4.534293,0.266881,"{'Fantasy': 4835, 'Young Adult': 4672, 'Scienc..."


In [7]:
cold_users_events_with_recs = \
    events_test[events_test["user_id"].isin(cold_users)] \
    .merge(top_k_pop_items, on="item_id", how="left")

cold_user_items_no_avg_rating_idx = cold_users_events_with_recs["avg_rating"].isnull()
cold_user_recs = cold_users_events_with_recs[~cold_user_items_no_avg_rating_idx] \
    [["user_id", "item_id", "rating", "avg_rating"]]

In [8]:
# Проверьте количество строк, чтобы убедиться, что оно осталось прежним
original_row_count = len(events_test[events_test["user_id"].isin(cold_users)])
current_row_count = len(cold_users_events_with_recs)

# Проверяем, одинаково ли количество строк
original_row_count, current_row_count

(9672, 9672)

In [9]:
# доли событий «холодных» пользователей в events_test рекомендации в top_k_pop_items
proportion_matched_recs = len(cold_user_recs) / len(cold_user_items_no_avg_rating_idx)
proportion_matched_recs_rounded = round(proportion_matched_recs, 2)
proportion_matched_recs_rounded

0.2

In [10]:
# посчитаем метрики рекомендаций
from sklearn.metrics import mean_squared_error, mean_absolute_error

rmse = mean_squared_error(cold_user_recs["rating"], cold_user_recs["avg_rating"], squared=False)
mae = mean_absolute_error(cold_user_recs["rating"], cold_user_recs["avg_rating"])
print(round(rmse, 2), round(mae, 2))

0.78 0.62


In [11]:
cold_users_hit_ratio = cold_users_events_with_recs.groupby("user_id").agg(hits=("avg_rating", lambda x: (~x.isnull()).mean()))

print(f"Доля пользователей без релевантных рекомендаций: {(cold_users_hit_ratio == 0).mean().iat[0]:.2f}")
print(f"Среднее покрытие пользователей: {cold_users_hit_ratio[cold_users_hit_ratio != 0].mean().iat[0]:.2f}")

Доля пользователей без релевантных рекомендаций: 0.59
Среднее покрытие пользователей: 0.44


In [12]:
# Посчитайте количество уникальных пользователей и предметов
num_users = len(events['user_id'].unique())
num_items = len(events['item_id'].unique())

# количество всех ячеек в матрице
num_cells = num_users * num_items

# количество пустых ячеек в матрице
col_null_cells = len(events[events['rating'] == 0])

# Посчитайте степень разреженности
sparsity = col_null_cells / num_cells

print('Степень разреженности U-I-матрицы: ', sparsity)

Степень разреженности U-I-матрицы:  0.0


### Модель SVD

In [ ]:
# pip install surprise

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [13]:
from surprise import Dataset, Reader
from surprise import SVD

# используем Reader из библиотеки surprise для преобразования событий (events)
# в формат, необходимый surprise
reader = Reader(rating_scale=(1, 5))
surprise_train_set = Dataset.load_from_df(events_train[['user_id', 'item_id', 'rating']], reader)
surprise_train_set = surprise_train_set.build_full_trainset()

# инициализируем модель
svd_model = SVD(n_factors=100, random_state=0)

# обучаем модель
svd_model.fit(surprise_train_set)

In [14]:
surprise_test_set = list(events_test[['user_id', 'item_id', 'rating']].itertuples(index=False))

# получаем рекомендации для тестовой выборки
svd_predictions = svd_model.test(surprise_test_set)

In [15]:
from surprise import accuracy

rmse = accuracy.rmse(svd_predictions)
mae = accuracy.mae(svd_predictions)
                     
print(rmse, mae)

RMSE: 0.8289
MAE:  0.6474
0.8288711689059135 0.647437483750257


In [16]:
from surprise import NormalPredictor

# инициализируем состояние генератора, это необходимо для получения
# одной и той же последовательности случайных чисел, только в учебных целях
np.random.seed(0)

random_model = NormalPredictor()

random_model.fit(surprise_train_set)
random_predictions = random_model.test(surprise_test_set)

In [19]:
rmse = accuracy.rmse(random_predictions)
mae = accuracy.mae(random_predictions)

RMSE: 1.2628
MAE:  1.0018


#### Получение рекомендаций

In [17]:
def get_recommendations_svd(user_id, all_items, events, model, include_seen=True, n=5):

    """ возвращает n рекомендаций для user_id """
    
    # получим список идентификаторов всех книг
    all_items = set(events['item_id'].unique())
        
    # учитываем флаг, стоит ли уже прочитанные книги включать в рекомендации
    if include_seen:
        items_to_predict = list(all_items)
    else:
        # получим список книг, которые пользователь уже прочитал ("видел")
        seen_items = set(events[events['user_id'] == user_id]['item_id'].unique())
        
        # книги, которые пользователь ещё не читал
        # только их и будем включать в рекомендации
        items_to_predict = list(all_items - seen_items)
    
    # получаем скоры для списка книг, т. е. рекомендации
    predictions = [model.predict(user_id, item_id) for item_id in items_to_predict]
    
    # сортируем рекомендации по убыванию скора и берём только n первых
    recommendations = sorted(predictions, key=lambda x: x.est, reverse=True)[:n]
    
    return pd.DataFrame([(pred.iid, pred.est) for pred in recommendations], columns=["item_id", "score"])

In [19]:
get_recommendations_svd(1296647, items, events_test, svd_model)

,item_id,score
0,7864312,4.981188
1,25793186,4.912001
2,12174312,4.898052
3,13208,4.894869
4,33353628,4.891661


In [18]:
# выберем произвольного пользователя из тренировочной выборки ("прошлого")
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "started_at", "read_at", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_svd(user_id, items, events_train, svd_model)
user_recommendations = user_recommendations.merge(items[["item_id", "author", "title", "genre_and_votes"]], on="item_id")
display(user_recommendations)

user_id: 1169023
История (последние события, recent)


,author,title,started_at,read_at,rating,genre_and_votes
68,Veronica Roth,"Divergent (Divergent, #1)",2014-06-02,2014-06-04,4,"{'Young Adult': 20260, 'Science Fiction-Dystop..."
69,"Gillian Flynn, В. Русанов",Gone Girl,2014-05-27,2014-05-29,5,"{'Fiction': 11773, 'Mystery': 9965, 'Thriller'..."
70,Kathy Reichs,"Death du Jour (Temperance Brennan, #2)",2014-05-24,2014-05-27,4,"{'Mystery': 1206, 'Mystery-Crime': 579, 'Ficti..."
71,Chelsea Cain,"Heartsick (Archie Sheridan & Gretchen Lowell, #1)",2014-05-22,2014-05-22,5,"{'Mystery': 832, 'Thriller': 653, 'Fiction': 4..."
72,"Jussi Adler-Olsen, Lisa Hartford","The Keeper of Lost Causes (Department Q, #1)",2014-05-30,2014-06-02,3,"{'Mystery': 1225, 'Mystery-Crime': 627, 'Ficti..."
73,Gillian Flynn,Dark Places,2014-05-17,2014-05-22,4,"{'Mystery': 4534, 'Fiction': 4055, 'Thriller':..."
74,Audrey Niffenegger,Her Fearful Symmetry,2014-05-05,2014-05-08,2,"{'Fiction': 1984, 'Fantasy': 674, 'Fantasy-Par..."
75,Kathy Reichs,"Déjà Dead (Temperance Brennan, #1)",2014-05-13,2014-05-17,4,"{'Mystery': 2141, 'Fiction': 904, 'Mystery-Cri..."
76,Carolyn Parkhurst,The Dogs of Babel,2014-05-09,2014-05-10,5,"{'Fiction': 522, 'Mystery': 102, 'Animals': 77..."
77,George R.R. Martin,"A Dance with Dragons (A Song of Ice and Fire, #5)",2014-05-04,2014-05-04,5,"{'Fantasy': 22247, 'Fiction': 4512, 'Fantasy-E..."


Рекомендации


,item_id,score,author,title,genre_and_votes
0,2199,5,Doris Kearns Goodwin,Team of Rivals: The Political Genius of Abraha...,"{'History': 4174, 'Nonfiction': 2127, 'Biograp..."
1,16255632,5,"David Gaider, Ben Gelinas, Mike Laidlaw, Dave ...",Dragon Age: The World of Thedas Volume 1,"{'Fantasy': 134, 'Games-Video Games': 28, 'Art..."
2,2363958,5,João Guimarães Rosa,Grande Sertão: Veredas,"{'Fiction': 85, 'Classics': 69, 'Cultural-Braz..."
3,22552026,5,Jason Reynolds,Long Way Down,"{'Young Adult': 1871, 'Poetry': 1737, 'Contemp..."
4,29237211,5,"Brian K. Vaughan, Fiona Staples","Saga, Vol. 7 (Saga, #7)","{'Sequential Art-Graphic Novels': 2539, 'Seque..."


# === Базовые подходы: коллаборативная фильтрация

In [19]:
import scipy
import sklearn.preprocessing

# перекодируем идентификаторы пользователей: 
# из имеющихся в последовательность 0, 1, 2, ...
user_encoder = sklearn.preprocessing.LabelEncoder()
user_encoder.fit(events["user_id"])
events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])

# перекодируем идентификаторы объектов: 
# из имеющихся в последовательность 0, 1, 2, ...
item_encoder = sklearn.preprocessing.LabelEncoder()
item_encoder.fit(items["item_id"])
items["item_id_enc"] = item_encoder.transform(items["item_id"])
events_train["item_id_enc"] = item_encoder.transform(events_train["item_id"])
events_test["item_id_enc"] = item_encoder.transform(events_test["item_id"])

/tmp/ipykernel_3702/1044897688.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["user_id_enc"] = user_encoder.transform(events_train["user_id"])
/tmp/ipykernel_3702/1044897688.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["user_id_enc"] = user_encoder.transform(events_test["user_id"])
/tmp/ipykernel_3702/1044897688.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t

In [20]:
events_train['item_id_enc'].max()

43304

In [21]:
# Посчитайте количество уникальных пользователей и предметов
num_users = len(events_train['user_id'].unique())
num_items = len(events_train['item_id'].unique())

# рассчитываем размер матрицы user_item_matrix_train в гигабайтах
num_cells = (num_users * num_items) / (1024 ** 3)

print('Размер матрицы в гигабайтах: ', num_cells)

Размер матрицы в гигабайтах:  16.54028546065092


In [22]:
# создаём sparse-матрицу формата CSR 
user_item_matrix_train = scipy.sparse.csr_matrix((
    events_train["rating"],
    (events_train['user_id_enc'], events_train['item_id_enc'])),
    dtype=np.int8)

In [23]:
import sys

# sparse-матрица формата CSR меньше занимает оперативной памяти
sum([sys.getsizeof(i) for i in user_item_matrix_train.data])/1024**3

0.26370687410235405

### Модель ALS

In [28]:
# pip install implicit

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [24]:
from implicit.als import AlternatingLeastSquares

als_model = AlternatingLeastSquares(factors=50, iterations=50, regularization=0.05, random_state=0)
als_model.fit(user_item_matrix_train)

/home/mle-user/mle_projects/mle-recsys-start/env_recsys_start/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/mle-user/mle_projects/mle-recsys-start/env_recsys_start/lib/python3.10/site-packages/implicit/cpu/als.py:95: RuntimeWarning: OpenBLAS is configured to use 4 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()
100%|██████████| 50/50 [04:39<00:00,  5.59s/it]


In [25]:
def get_recommendations_als(user_item_matrix, model, user_id, user_encoder, item_encoder, include_seen=True, n=5):
    """
    Возвращает отранжированные рекомендации для заданного пользователя
    """
    user_id_enc = user_encoder.transform([user_id])[0]
    recommendations = model.recommend(
         user_id_enc, 
         user_item_matrix[user_id_enc], 
         filter_already_liked_items=not include_seen,
         N=n)
    recommendations = pd.DataFrame({"item_id_enc": recommendations[0], "score": recommendations[1]})
    recommendations["item_id"] = item_encoder.inverse_transform(recommendations["item_id_enc"])
    
    return recommendations

In [26]:
# Выбор произвольного пользователя из тренировочной выборки
user_id = events_train['user_id'].sample().iat[0]

print(f"user_id: {user_id}")

print("История (последние события, recent)")
user_history = (
    events_train
    .query("user_id == @user_id")
    .merge(items.set_index("item_id")[["author", "title", "genre_and_votes"]], on="item_id")
)
user_history_to_print = user_history[["author", "title", "rating", "genre_and_votes"]].tail(10)
display(user_history_to_print)

print("Рекомендации")
user_recommendations = get_recommendations_als(user_item_matrix_train, als_model, user_id, user_encoder, item_encoder, include_seen=False, n=5)
user_recommendations = user_recommendations.merge(items, on="item_id")
user_recommendations["seen"] = user_recommendations["item_id"].isin(user_history["item_id"])
display(user_recommendations)

user_id: 1012012
История (последние события, recent)


,author,title,rating,genre_and_votes
40,Nick Cave,The Death of Bunny Munro,5,"{'Fiction': 340, 'Thriller': 55, 'Contemporary..."
41,David Byrne,How Music Works,4,"{'Music': 1012, 'Nonfiction': 554, 'History': ..."
42,"Haruki Murakami, Alfred Birnbaum, Philip Gabriel",Underground: The Tokyo Gas Attack and the Japa...,4,"{'Nonfiction': 785, 'Cultural-Japan': 336, 'Hi..."
43,Jennifer Egan,A Visit from the Goon Squad,4,"{'Fiction': 3761, 'Contemporary': 543, 'Music'..."
44,Oscar Wilde,The Happy Prince,4,"{'Classics': 552, 'Short Stories': 273, 'Ficti..."
45,"Roald Dahl, Quentin Blake",The Twits,3,"{'Childrens': 1359, 'Fiction': 939, 'Fantasy':..."
46,Kazuo Ishiguro,The Remains of the Day,3,"{'Fiction': 4784, 'Historical-Historical Ficti..."
47,Aldous Huxley,Brave New World,3,"{'Classics': 17717, 'Fiction': 12564, 'Science..."
48,Kurt Vonnegut Jr.,Slaughterhouse-Five,2,"{'Classics': 12568, 'Fiction': 11930, 'Science..."
49,John Steinbeck,Of Mice and Men,4,"{'Classics': 25191, 'Fiction': 12140, 'Academi..."


Рекомендации


,item_id_enc_x,score,item_id,author,title,description,genre_and_votes,num_pages,average_rating,ratings_count,...,country_code,language_code,format,is_ebook,isbn,isbn13,genre_and_votes_dict,genre_and_votes_str,item_id_enc_y,seen
0,35810,0.535703,18007564,Andy Weir,The Martian,"Six days ago, astronaut Mark Watney became one...","{'Science Fiction': 11966, 'Fiction': 8430}",369,4.39,435440,...,US,eng,Hardcover,False,0804139024,9780804139021,"{'Academic': None, 'Academic-Academia': None, ...","Science Fiction 11966, Fiction 8430",35810,False
1,7,0.527021,11,Douglas Adams,The Hitchhiker's Guide to the Galaxy (Hitchhik...,Seconds before the Earth is demolished to make...,"{'Science Fiction': 13941, 'Fiction': 9853, 'H...",216,4.20,940154,...,US,en-US,Mass Market Paperback,False,0345391802,9780345391803,"{'Academic': None, 'Academic-Academia': None, ...","Science Fiction 13941, Fiction 9853, Humor 534...",7,False
2,493,0.457784,4671,"F. Scott Fitzgerald, Francis Scott Fitzgerald",The Great Gatsby,"THE GREAT GATSBY, F. Scott Fitzgerald's third ...","{'Classics': 42010, 'Fiction': 19710, 'Academi...",180,3.89,2758812,...,US,eng,Paperback,False,0743273567,9780743273565,"{'Academic': None, 'Academic-Academia': None, ...","Classics 42010, Fiction 19710, Academic-School...",493,False
3,8840,0.438138,157993,"Antoine de Saint-Exupéry, Richard Howard, Ivan...",The Little Prince,"Moral allegory and spiritual autobiography, Th...","{'Classics': 13703, 'Fiction': 7638, 'Fantasy'...",93,4.28,763309,...,US,eng,Paperback,False,0156012197,9780156012195,"{'Academic': None, 'Academic-Academia': None, ...","Classics 13703, Fiction 7638, Fantasy 5162, Ch...",8840,False
4,884,0.427312,7613,"George Orwell, Boris Grabnar, Peter Škerl",Animal Farm,As ferociously fresh as it was more than a hal...,"{'Classics': 30122, 'Fiction': 17416, 'Science...",122,3.88,1928931,...,US,eng,Paperback,False,0452284244,9780452284241,"{'Academic': None, 'Academic-Academia': None, ...","Classics 30122, Fiction 17416, Science Fiction...",884,False


In [27]:
# получаем список всех возможных user_id (перекодированных)
user_ids_encoded = range(len(user_encoder.classes_))

# получаем рекомендации для всех пользователей
als_recommendations = als_model.recommend(
    user_ids_encoded, 
    user_item_matrix_train[user_ids_encoded], 
    filter_already_liked_items=False, N=100)

In [28]:
# преобразуем полученные рекомендации в табличный формат
item_ids_enc = als_recommendations[0]
als_scores = als_recommendations[1]

als_recommendations = pd.DataFrame({
    "user_id_enc": user_ids_encoded,
    "item_id_enc": item_ids_enc.tolist(), 
    "score": als_scores.tolist()})
als_recommendations = als_recommendations.explode(["item_id_enc", "score"], ignore_index=True)

# приводим типы данных
als_recommendations["item_id_enc"] = als_recommendations["item_id_enc"].astype("int")
als_recommendations["score"] = als_recommendations["score"].astype("float")

# получаем изначальные идентификаторы
als_recommendations["user_id"] = user_encoder.inverse_transform(als_recommendations["user_id_enc"])
als_recommendations["item_id"] = item_encoder.inverse_transform(als_recommendations["item_id_enc"])
als_recommendations = als_recommendations.drop(columns=["user_id_enc", "item_id_enc"])

In [29]:
als_recommendations = als_recommendations[["user_id", "item_id", "score"]]
als_recommendations.to_parquet("als_recommendations.parquet")

In [30]:
# Для удобства оценки добавим в датафрейм с рекомендациями истинные оценки из тестовой выборки:
als_recommendations = (
    als_recommendations
    .merge(events_test[["user_id", "item_id", "rating"]]
               .rename(columns={"rating": "rating_test"}), 
           on=["user_id", "item_id"], how="left")
)

In [31]:
# Подсчитать метрику NDCG для одного пользователя
import sklearn.metrics

def compute_ndcg(rating: pd.Series, score: pd.Series, k):

    """ подсчёт ndcg
    rating: истинные оценки
    score: оценки модели
    k: количество айтемов (по убыванию score) для оценки, остальные - отбрасываются
    """
    
    # если кол-во объектов меньше 2, то NDCG - не определена
    if len(rating) < 2:
        return np.nan

    ndcg = sklearn.metrics.ndcg_score(np.asarray([rating.to_numpy()]), np.asarray([score.to_numpy()]), k=k)

    return ndcg

Умея считать NDCG для одного пользователя, посчитаем данную метрику, например, для
k=5 для всех пользователей из тестовой выборки. В результате каждому пользователю будет соответствовать одно значение NDCG@5. Запись “NDCG@5” означает, что метрика NDCG считается для пяти объектов. 

In [32]:
rating_test_idx = ~als_recommendations["rating_test"].isnull()
ndcg_at_5_scores = als_recommendations[rating_test_idx].groupby("user_id").apply(lambda x: compute_ndcg(x["rating_test"], x["score"], k=5))

# Имея ряд значений NDCG@5 по пользователям, можно посчитать среднее её значение (по всем пользователям)
print('Среднее значение NDCG@5: ', round(ndcg_at_5_scores.mean(), 2))

Среднее значение NDCG@5:  0.98


In [33]:
# Оценка доли пользователей, для которых удалось посчитать метрику NDCG
total_users = len(user_encoder.classes_)
users_ndcg = ndcg_at_5_scores.notnull().sum()
users_ndcg_proportion = users_ndcg / total_users

round(users_ndcg_proportion, 2)

0.04

##### Используем модель ALS для получения похожих items с помощью метода similar_items

In [34]:
# Предположим, что item_ids для анализа уже определены
item_ids_to_analyze = [100, 200, 300]  # пример ID айтемов, которые будем анализировать

# Используем модель для получения похожих айтемов
similar_items_dict = {}
for item_id in item_ids_to_analyze:
    similar_items = als_model.similar_items(item_id, N=10)
    similar_items_dict[item_id] = similar_items

# Преобразуем результаты в DataFrame
similar_items_list = []
for item_id, (sim_item_ids, scores) in similar_items_dict.items():
    for sim_item_id, score in zip(sim_item_ids, scores):
        similar_items_list.append((item_id, sim_item_id, score))

similar_items_df = pd.DataFrame(similar_items_list, columns=["item_id", "similar_item_id", "score"])

similar_items_df

,item_id,similar_item_id,score
0,100,100,1.000000
1,100,416,0.955224
2,100,8081,0.945551
3,100,240,0.940132
4,100,2962,0.935671
5,100,1479,0.932158
6,100,8742,0.932129
7,100,1817,0.926266
8,100,6934,0.925741
9,100,3335,0.925313


##### Используем модель ALS для получения похожих users с помощью метода similar_users

In [35]:
# Выберем несколько users для анализа
users_ids_to_analyze = [1, 50, 100]  # Пример ID user, которые будем анализировать

# Используем модель для получения похожих user
similar_users_dict = {}
for user_id in users_ids_to_analyze:
    similar_user = als_model.similar_users(user_id, N=10)
    similar_users_dict[user_id] = similar_user

# Преобразуем результаты в DataFrame
similar_users_list = []
for user_id, (sim_user_ids, scores) in similar_users_dict.items():
    for sim_user_id, score in zip(sim_user_ids, scores):
        similar_users_list.append((user_id, sim_user_id, score))

similar_users_df = pd.DataFrame(similar_users_list, columns=["user_id", "similar_user_id", "score"])

similar_users_df

,user_id,similar_user_id,score
0,1,1,1.000000
1,1,118312,0.994112
2,1,199361,0.993490
3,1,205072,0.992585
4,1,343231,0.992469
5,1,29103,0.992195
6,1,105530,0.991213
7,1,313396,0.990874
8,1,30087,0.990061
9,1,272370,0.989844


# === Базовые подходы: контентные рекомендации

In [36]:
items["genre_and_votes"] = items["genre_and_votes"].apply(eval)

In [37]:
def get_genres(items):

    """ 
    извлекает список жанров по всем книгам, 
    подсчитывает долю голосов по каждому их них
    """
    
    genres_counter = {}
    
    for k, v, in items.iterrows():
        genre_and_votes = v["genre_and_votes"]
        if genre_and_votes is None or not isinstance(genre_and_votes, dict):
            continue
        for genre, votes in genre_and_votes.items():
            # увеличиваем счётчик жанров
            try:
                genres_counter[genre] += votes
            except KeyError:
                genres_counter[genre] = 0

    genres = pd.Series(genres_counter, name="votes")
    genres = genres.to_frame()
    genres = genres.reset_index().rename(columns={"index": "name"})
    genres.index.name = "genre_id"
    
    return genres
   
genres = get_genres(items)

In [38]:
genres["score"] = genres["votes"] / genres["votes"].sum()
genres.sort_values(by="score", ascending=False).head(10)

,name,votes,score
genre_id,,,
25,Fantasy,6850060,0.149651
1,Fiction,6406256,0.139955
38,Classics,3414934,0.074605
18,Young Adult,3296951,0.072027
34,Romance,2422614,0.052926
5,Nonfiction,1737406,0.037957
16,Historical-Historical Fiction,1531205,0.033452
20,Mystery,1371196,0.029956
24,Science Fiction,1218917,0.026629


Построим матрицу вида «книга-жанр»

In [39]:
def get_item2genre_matrix(genres, items):

    genre_names_to_id = genres.reset_index().set_index("name")["genre_id"].to_dict()
    
    # list to build CSR matrix
    genres_csr_data = []
    genres_csr_row_idx = []
    genres_csr_col_idx = []
    
    for item_idx, (k, v) in enumerate(items.iterrows()):
        if v["genre_and_votes"] is None:
            continue
        for genre_name, votes in v["genre_and_votes"].items():
            genre_idx = genre_names_to_id[genre_name]
            genres_csr_data.append(int(votes))
            genres_csr_row_idx.append(item_idx)
            genres_csr_col_idx.append(genre_idx)

    genres_csr = scipy.sparse.csr_matrix((genres_csr_data, (genres_csr_row_idx, genres_csr_col_idx)), shape=(len(items), len(genres)))
    # нормализуем, чтобы сумма оценок принадлежности к жанру была равна 1
    genres_csr = sklearn.preprocessing.normalize(genres_csr, norm='l1', axis=1)
    
    return genres_csr

In [40]:
items = items.sort_values(by="item_id_enc")
all_items_genres_csr = get_item2genre_matrix(genres, items)

In [41]:
user_id = 1000010
user_events = events_train.query("user_id == @user_id")[["item_id", "rating"]]
user_items = items[items["item_id"].isin(user_events["item_id"])]

user_items_genres_csr = get_item2genre_matrix(genres, user_items)
user_items_genres_csr

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 149 stored elements and shape (22, 815)>

In [47]:
user_items_genres_csr.count_nonzero()

149

In [48]:
# вычислим склонность пользователя к жанрам как среднее взвешенное значение популяции на его оценки книг.

# преобразуем пользовательские оценки из списка в вектор-столбец
user_ratings = user_events["rating"].to_numpy() / 5
user_ratings = np.expand_dims(user_ratings, axis=1)

user_items_genres_weighted = user_items_genres_csr.multiply(user_ratings)

user_genres_scores = np.asarray(user_items_genres_weighted.mean(axis=0))

In [49]:
# выведем список жанров, которые предпочитает пользователь

user_genres = genres.copy()
user_genres["score"] = np.ravel(user_genres_scores)
user_genres = user_genres[user_genres["score"] > 0].sort_values(by=["score"], ascending=False)

user_genres.head(5)

,name,votes,score
genre_id,,,
1,Fiction,6406256,0.185241
38,Classics,3414934,0.103879
25,Fantasy,6850060,0.072447
5,Nonfiction,1737406,0.050865
24,Science Fiction,1218917,0.040920


Получите наиболее релевантные рекомендации для пользователя

In [50]:
from sklearn.metrics.pairwise import cosine_similarity

# вычисляем сходство между вектором пользователя и векторами по книгам
similarity_scores = cosine_similarity(all_items_genres_csr, user_genres_scores)

# преобразуем в одномерный массив
similarity_scores = similarity_scores.flatten()

# получаем индексы top-k (по убыванию значений), по сути, индексы книг (encoded)
k = 5
top_k_indices = np.argsort(similarity_scores)[-k:]

In [51]:
selected_items = items[items["item_id_enc"].isin(top_k_indices)]

with pd.option_context("max_colwidth", 100):
   display(selected_items[["author", "title", "genre_and_votes"]]) 

,author,title,genre_and_votes
80465,G.K. Chesterton,The Napoleon of Notting Hill,"{'Fiction': 166, 'Classics': 88, 'Fantasy': 44, 'Humor': 22, 'Literature': 20}"
1168335,Ray Bradbury,"Dandelion Wine (Green Town, #1)","{'Fiction': 1438, 'Classics': 914, 'Science Fiction': 529, 'Fantasy': 456, 'Young Adult': 212}"
393210,"G.K. Chesterton, Jonathan Lethem",The Man Who Was Thursday: A Nightmare,"{'Fiction': 1257, 'Classics': 929, 'Mystery': 469, 'Fantasy': 293, 'Philosophy': 156, 'Literatur..."
2244467,Samuel Butler,"Erewhon (Erewhon , #1)","{'Fiction': 162, 'Classics': 139, 'Science Fiction': 60, 'Fantasy': 55}"
39408,"Paulo Coelho, Alan R. Clarke, James Noel Smith",The Alchemist,"{'Fiction': 14023, 'Classics': 5787, 'Fantasy': 3289, 'Philosophy': 2759}"


# === Базовые подходы: валидация

In [61]:
def process_events_recs_for_binary_metrics(events_train, events_test, recs, top_k=None):

    """
    размечает пары <user_id, item_id> для общего множества пользователей признаками
    - gt (ground truth)
    - pr (prediction)
    top_k: расчёт ведётся только для top k-рекомендаций
    """

    events_test["gt"] = True
    common_users = set(events_test["user_id"]) & set(recs["user_id"])

    print(f"Common users: {len(common_users)}")
    
    events_for_common_users = events_test[events_test["user_id"].isin(common_users)].copy()
    recs_for_common_users = recs[recs["user_id"].isin(common_users)].copy()

    recs_for_common_users = recs_for_common_users.sort_values(["user_id", "score"], ascending=[True, False])

    # оставляет только те item_id, которые были в events_train, 
    # т. к. модель не имела никакой возможности давать рекомендации для новых айтемов
    events_for_common_users = events_for_common_users[events_for_common_users["item_id"].isin(events_train["item_id"].unique())]

    if top_k is not None:
        recs_for_common_users = recs_for_common_users.groupby("user_id").head(top_k)
    
    events_recs_common = events_for_common_users[["user_id", "item_id", "gt"]].merge(
        recs_for_common_users[["user_id", "item_id", "score"]], 
        on=["user_id", "item_id"], how="outer")    

    events_recs_common["gt"] = events_recs_common["gt"].fillna(False)
    events_recs_common["pr"] = ~events_recs_common["score"].isnull()
    
    events_recs_common["tp"] = events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fp"] = ~events_recs_common["gt"] & events_recs_common["pr"]
    events_recs_common["fn"] = events_recs_common["gt"] & ~events_recs_common["pr"]

    return events_recs_common

In [53]:
events_recs_for_binary_metrics = process_events_recs_for_binary_metrics(
  events_train,
    events_test, 
    als_recommendations, 
    top_k=10)

/tmp/ipykernel_61273/3166051426.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_test["gt"] = True


Common users: 123223


In [62]:
def compute_cls_metrics(events_recs_for_binary_metric):
    
    groupper = events_recs_for_binary_metric.groupby("user_id")

    # precision = tp / (tp + fp)
    precision = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fp"].sum())
    precision = precision.fillna(0).mean()
    
    # recall = tp / (tp + fn)
    recall = groupper["tp"].sum()/(groupper["tp"].sum()+groupper["fn"].sum())
    recall = recall.fillna(0).mean()

    return precision, recall

In [55]:
precision, recall = compute_cls_metrics(events_recs_for_binary_metrics)
precision, round(recall, 3)

(0.008732947582837622, 0.031)

In [56]:
precision, recall = compute_cls_metrics(events_recs_for_binary_metrics)
precision, round(recall, 3)

(0.008732947582837622, 0.031)

# === Двухстадийный подход: метрики

In [57]:
# Загрузка данных
als_recommendations = pd.read_parquet("als_recommendations.parquet")

In [58]:
# Количество уникальных объектов в рекомендациях
recom_uniq = len(als_recommendations['item_id'].unique())

# Общее количество объектов
total_items = len(items['item_id'].unique())

# расчёт покрытия по объектам
cov_items = recom_uniq/total_items
print(f"{cov_items:.2f}") 

0.09


In [59]:
# разметим каждую рекомендацию признаком read
events_train["read"] = True
als_recommendations = als_recommendations.merge(events_train, on=["user_id", "item_id"], how="left")
als_recommendations["read"] = als_recommendations["read"].fillna(False).astype("bool")

# проставим ранги
als_recommendations = als_recommendations.sort_values(by=['user_id', 'score'], ascending=[True, False])
als_recommendations["rank"] = als_recommendations.groupby("user_id").cumcount() + 1

# посчитаем novelty по пользователям
novelty_5 = (1-als_recommendations.query("rank <= 5").groupby("user_id")["read"].mean())

# посчитаем средний novelty
print('Cредний Novelty@5: ', round(novelty_5.mean(),2))

/tmp/ipykernel_61273/3130406217.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  events_train["read"] = True


Cредний Novelty@5:  0.61


# === Двухстадийный подход: модель

In [43]:
# задаём точку разбиения
split_date_for_labels = pd.to_datetime("2017-09-15").date()

split_date_for_labels_idx = events_test["started_at"] < split_date_for_labels
events_labels = events_test[split_date_for_labels_idx].copy()
events_test_2 = events_test[~split_date_for_labels_idx].copy()

In [44]:
events_labels.nunique()

user_id             99849
item_id             22003
started_at             45
read_at                92
is_read                 1
rating                  5
is_reviewed             2
started_at_month        2
user_id_enc         99849
item_id_enc         22003
dtype: int64

In [45]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations = pd.read_parquet("candidates/training/als_recommendations.parquet")
content_recommendations = pd.read_parquet("candidates/training/content_recommendations.parquet")

candidates = pd.merge(
    als_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer")

In [63]:
candidates

,user_id,item_id,als_score,cnt_score
0,1000000,3,0.972557,0.920225
1,1000000,15881,0.890201,0.905740
2,1000000,5,0.865850,0.918026
3,1000000,6,0.834282,0.916345
4,1000000,2,0.792929,0.925806
...,...,...,...,...
82993089,1430584,31327371,NaN,0.786363
82993090,1430584,32841355,NaN,0.784905
82993091,1430584,33828743,NaN,0.784706
82993092,1430584,34037113,NaN,0.784556


In [46]:
# добавляем таргет к кандидатам со значением:
# — 1 для тех item_id, которые пользователь прочитал
# — 0, для всех остальных 

events_labels["target"] = 1
candidates = candidates.merge(events_labels[["user_id", "item_id", "target"]], 
                              on=["user_id", "item_id"], how="left")
candidates["target"] = candidates["target"].fillna(0).astype("int")

# в кандидатах оставляем только тех пользователей, у которых есть хотя бы один положительный таргет
candidates_to_sample = candidates.groupby("user_id").filter(lambda x: x["target"].sum() > 0)

# для каждого пользователя оставляем только 4 негативных примера
negatives_per_user = 4
candidates_for_train = pd.concat([
    candidates_to_sample.query("target == 1"),
    candidates_to_sample.query("target == 0") \
        .groupby("user_id") \
        .apply(lambda x: x.sample(negatives_per_user, random_state=0))
    ])

In [47]:
candidates_for_train

,user_id,item_id,als_score,cnt_score,target
615,1000006,29868610,0.286715,NaN,1
632,1000006,7445,0.230529,NaN,1
649,1000006,18812405,0.178382,NaN,1
1998,1000019,37415,0.043595,NaN,1
2302,1000023,7260188,0.598791,NaN,1
...,...,...,...,...,...
"(1430579, 82992597)",1430579,15698462,NaN,0.900922,0
"(1430584, 43058418)",1430584,18774964,0.222126,NaN,0
"(1430584, 82993064)",1430584,8393104,NaN,0.795215,0
"(1430584, 82993001)",1430584,24929,NaN,0.847833,0


### Catboost

In [49]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score']
target = 'target'

# Create the Pool object
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6526057	total: 88.8ms	remaining: 1m 28s
100:	learn: 0.5118959	total: 2.48s	remaining: 22.1s
200:	learn: 0.5111710	total: 4.93s	remaining: 19.6s
300:	learn: 0.5105208	total: 8.41s	remaining: 19.5s
400:	learn: 0.5100174	total: 10.7s	remaining: 16s
500:	learn: 0.5095747	total: 13.3s	remaining: 13.3s
600:	learn: 0.5091600	total: 15.9s	remaining: 10.6s
700:	learn: 0.5087803	total: 18.5s	remaining: 7.88s
800:	learn: 0.5084220	total: 20.8s	remaining: 5.17s
900:	learn: 0.5080930	total: 23.4s	remaining: 2.57s
999:	learn: 0.5078081	total: 25.7s	remaining: 0us


In [50]:
# загружаем рекомендации от двух базовых генераторов
als_recommendations_2 = pd.read_parquet("candidates/inference/als_recommendations.parquet")
content_recommendations_2 = pd.read_parquet("candidates/inference/content_recommendations.parquet")

candidates_to_rank = pd.merge(als_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "als_score"}),
    content_recommendations_2[["user_id", "item_id", "score"]].rename(columns={"score": "cnt_score"}),
    on=["user_id", "item_id"],
    how="outer")

# оставляем только тех пользователей, что есть в тестовой выборке, для экономии ресурсов
candidates_to_rank = candidates_to_rank[candidates_to_rank["user_id"].isin(events_test_2["user_id"].drop_duplicates())]
print(len(candidates_to_rank))

14517152


In [51]:
inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

# для каждого пользователя проставляем rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1

max_recommendations_per_user = 100
final_recommendations = candidates_to_rank[candidates_to_rank["rank"] <= max_recommendations_per_user]

In [24]:
# Выводим результат
print(final_recommendations)

          user_id   item_id  als_score  cnt_score  cb_score  rank
347       1000003     49628   0.446143   0.906649  0.583617     1
300       1000003   7260188   1.129979        NaN  0.509005     2
301       1000003   6148028   1.123475        NaN  0.509005     3
302       1000003   2767052   1.112699        NaN  0.509005     4
320       1000003     43641   0.617602        NaN  0.477032     5
...           ...       ...        ...        ...       ...   ...
43058095  1430580  23705512   0.016477        NaN  0.232760    96
43058096  1430580   6314763   0.016404        NaN  0.232760    97
43058097  1430580  11710373   0.016035        NaN  0.221228    98
43058098  1430580      7445   0.015793        NaN  0.221228    99
43058099  1430580  17802724   0.015771        NaN  0.221228   100

[7519400 rows x 6 columns]


In [52]:
events_inference = pd.concat([events_train, events_labels])

cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2,
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

NameError: name 'process_events_recs_for_binary_metrics' is not defined

# === Двухстадийный подход: построение признаков

In [53]:
items["age"] = 2018 - items["publication_year"]
invalid_age_idx = items["age"] < 0
items.loc[invalid_age_idx, "age"] = np.nan
items["age"] = items["age"].astype("float")

candidates_for_train = candidates_for_train.merge(items[["item_id", "age", "average_rating"]], on="item_id", how="left")
candidates_to_rank = candidates_to_rank.merge(items[["item_id", "age", "average_rating"]], on="item_id", how="left")

In [ ]:
candidates_to_rank['age'].median()

7.0

Используя события в events_train и events_inference, посчитайте и добавьте признаки пользователей к кандидатам в candidates_for_train и candidates_to_rank соответственно:
reading_years — длительность истории пользователя,
books_read — количество книг, прочитанных за всё время,
books_per_year — среднее количество прочитанных книг в год,
rating_avg — средняя оценка,
rating_std — дисперсия оценок.

In [54]:
def get_user_features(events):
    """ считает пользовательские признаки """
    
    user_features = events.groupby("user_id").agg(
        reading_years=("started_at", lambda x: (x.max()-x.min()).days/365.25),
        books_read=("item_id", "count"),
        rating_avg=("rating", "mean"),
        rating_std=("rating", "std"))
    
    user_features["books_per_year"] = user_features["books_read"] / user_features["reading_years"]
    
    return user_features
    
user_features_for_train = get_user_features(events_train)
candidates_for_train = candidates_for_train.merge(user_features_for_train, on="user_id", how="left")
  
# оставим только тех пользователей, что есть в тесте, для экономии ресурсов
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test["user_id"].drop_duplicates())]

user_features_for_ranking = get_user_features(events_inference)
candidates_to_rank = candidates_to_rank.merge(user_features_for_ranking, on="user_id", how="left")

In [16]:
candidates_for_train["books_read"].median()

32.0

Используя истории events_train и events_inference, а также ранее полученные артефакты по жанрам книг — словарь жанров genres, оценки книг по жанрам all_items_genres_csr — добавьте парные признаки, по одному на каждый жанр, которые совместно показывают, какие жанры предпочитает пользователь. 
Жанровость в данном случае — численный коэффициент принадлежности книги к жанру. Например, если пользователь прочитал три книги, которые с весами 0.3, 0.2, 0.4 из  all_items_genres_csr относятся к Fantasy, то интерес пользователя к Fantasy составляет среднее этих трёх оценок — 0.3.
Для экономии ресурсов возьмём не все жанры, а десять наиболее популярных. Все остальные отметим как не вошедшие в топ и обозначим как others. 

In [55]:
# определяем индексы топ-10 жанров и всех остальных
genres_top_k = 10
genres_top_idx = genres.sort_values("votes", ascending=False).head(genres_top_k).index
genres_others_idx = list(set(genres.index) - set(genres_top_idx))

genres_top_columns = [f"genre_{id}" for id in genres_top_idx]
genres_others_column = "genre_others"
genre_columns = genres_top_columns + [genres_others_column]

# составляем таблицу принадлежности книг к жанрам
item_genres = (
    pd.concat([
        # топ жанров
        pd.DataFrame(all_items_genres_csr[:, genres_top_idx].toarray(), columns=genres_top_columns),
        # все остальные жанры
        pd.DataFrame(all_items_genres_csr[:, genres_others_idx].sum(axis=1), columns=[genres_others_column])
        ],
        axis=1)
    .reset_index()
    .rename(columns={"index": "item_id_enc"})
)

# объединяем информацию принадлежности книг к жанрам с основной информацией о книгах
items = items.merge(item_genres, on="item_id_enc", how="left")

def get_user_genres(events, items, item_genre_columns):
    user_genres = (
        events
        .merge(items[["item_id"] + item_genre_columns], on="item_id", how="left")
        .groupby("user_id")[item_genre_columns].mean()
    )
    return user_genres
    
user_genres_for_train = get_user_genres(events_train, items, genre_columns)
candidates_for_train = candidates_for_train.merge(user_genres_for_train, on="user_id", how="left")

user_genres_for_ranking = get_user_genres(events_inference, items, genre_columns)
candidates_to_rank = candidates_to_rank.merge(user_genres_for_ranking, on="user_id", how="left")

In [74]:
genres.sort_values("votes", ascending=False).head(genres_top_k)

,name,votes,score
genre_id,,,
25,Fantasy,6850060,0.149651
1,Fiction,6406256,0.139955
38,Classics,3414934,0.074605
18,Young Adult,3296951,0.072027
34,Romance,2422614,0.052926
5,Nonfiction,1737406,0.037957
16,Historical-Historical Fiction,1531205,0.033452
20,Mystery,1371196,0.029956
24,Science Fiction,1218917,0.026629


In [77]:
print('Медиана жанровости книг Romance: ', round(candidates_to_rank['genre_34'].median(),2))

Медиана жанровости книг Romance:  0.04


#### Обучение и получение рекомендаций

Вы добавили в candidates_for_train и candidates_to_rank различные признаки. Обучите новую ранжирующую модель, которая их будет учитывать.

In [56]:
from catboost import CatBoostClassifier, Pool

# задаём имена колонок признаков и таргета
features = ['als_score', 'cnt_score', 
    'age', 'average_rating', 'reading_years', 'books_read', 
    'rating_avg', 'rating_std', 
    'books_per_year'] + genre_columns
target = 'target'

# создаём Pool
train_data = Pool(
    data=candidates_for_train[features], 
    label=candidates_for_train[target])

# инициализируем модель CatBoostClassifier
cb_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function='Logloss',
    verbose=100,
    random_seed=0,
)

# тренируем модель
cb_model.fit(train_data)

0:	learn: 0.6485043	total: 24.4ms	remaining: 24.4s
100:	learn: 0.4665268	total: 3.89s	remaining: 34.7s
200:	learn: 0.4578610	total: 7.46s	remaining: 29.7s
300:	learn: 0.4518428	total: 11.3s	remaining: 26.1s
400:	learn: 0.4471687	total: 14.4s	remaining: 21.5s
500:	learn: 0.4429055	total: 18.1s	remaining: 18s
600:	learn: 0.4390647	total: 21.5s	remaining: 14.3s
700:	learn: 0.4355651	total: 24.3s	remaining: 10.3s
800:	learn: 0.4321506	total: 27.9s	remaining: 6.93s
900:	learn: 0.4288350	total: 33.1s	remaining: 3.63s
999:	learn: 0.4257325	total: 37.5s	remaining: 0us


In [57]:
candidates_for_train[features]

,als_score,cnt_score,age,average_rating,reading_years,books_read,rating_avg,rating_std,books_per_year,genre_25,genre_1,genre_38,genre_18,genre_34,genre_5,genre_16,genre_20,genre_24,genre_33,genre_others
0,0.286715,NaN,NaN,3.90,1.820671,17.0,4.294118,0.685994,9.337218,0.180849,0.246138,0.105182,0.057684,0.010375,0.078927,0.004294,0.021665,0.008603,0.000000,0.286282
1,0.230529,NaN,12.0,4.24,1.820671,17.0,4.294118,0.685994,9.337218,0.180849,0.246138,0.105182,0.057684,0.010375,0.078927,0.004294,0.021665,0.008603,0.000000,0.286282
2,0.178382,NaN,4.0,3.81,1.820671,17.0,4.294118,0.685994,9.337218,0.180849,0.246138,0.105182,0.057684,0.010375,0.078927,0.004294,0.021665,0.008603,0.000000,0.286282
3,0.043595,NaN,12.0,3.87,0.276523,6.0,4.166667,1.169045,21.698020,0.049632,0.158224,0.000000,0.000000,0.000000,0.195082,0.000000,0.000000,0.082617,0.000000,0.514445
4,0.598791,NaN,8.0,4.03,0.005476,2.0,3.500000,0.707107,365.250000,0.318124,0.170366,0.000000,0.159612,0.019622,0.000000,0.000000,0.000000,0.077326,0.000000,0.254950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
213703,NaN,0.900922,6.0,3.60,6.499658,264.0,3.928030,1.143061,40.617523,0.176929,0.069304,0.007392,0.271108,0.080489,0.017241,0.007644,0.027871,0.021434,0.020003,0.300587
213704,0.222126,NaN,4.0,4.35,2.899384,11.0,3.454545,1.128152,3.793909,0.157407,0.224713,0.017886,0.061621,0.047617,0.084946,0.020774,0.041339,0.002553,0.001796,0.339348
213705,NaN,0.795215,8.0,3.64,2.899384,11.0,3.454545,1.128152,3.793909,0.157407,0.224713,0.017886,0.061621,0.047617,0.084946,0.020774,0.041339,0.002553,0.001796,0.339348
213706,NaN,0.847833,16.0,2.81,2.899384,11.0,3.454545,1.128152,3.793909,0.157407,0.224713,0.017886,0.061621,0.047617,0.084946,0.020774,0.041339,0.002553,0.001796,0.339348


Получите топ-100 самых релевантных рекомендация для каждого пользователя, используя обученную модель.

In [58]:
inference_data = Pool(data=candidates_to_rank[features])
predictions = cb_model.predict_proba(inference_data)

candidates_to_rank["cb_score"] = predictions[:, 1]

# для каждого пользователя проставим rank, начиная с 1 — это максимальный cb_score
candidates_to_rank = candidates_to_rank.sort_values(["user_id", "cb_score"], ascending=[True, False])
candidates_to_rank["rank"] = candidates_to_rank.groupby("user_id").cumcount() + 1

max_recommendations_per_user = 100
final_recommendations = candidates_to_rank.query("rank <= @max_recommendations_per_user")

In [59]:
final_recommendations['user_id'].nunique()

75194

In [60]:
final_recommendations.to_parquet("final_recommendations_feat.parquet")

In [63]:
# для экономии ресурсов оставим события только тех пользователей, 
# для которых следует оценить рекомендации
events_inference = pd.concat([events_train, events_labels])
events_inference = events_inference[events_inference["user_id"].isin(events_test_2["user_id"].drop_duplicates())]

cb_events_recs_for_binary_metrics_5 = process_events_recs_for_binary_metrics(
    events_inference,
    events_test_2,
    final_recommendations.rename(columns={"cb_score": "score"}), 
    top_k=5)

cb_precision_5, cb_recall_5 = compute_cls_metrics(cb_events_recs_for_binary_metrics_5)

print(f"precision: {cb_precision_5:.3f}, recall: {cb_recall_5:.3f}")

Common users: 75194
precision: 0.011, recall: 0.030


In [64]:
feature_importance = pd.DataFrame(cb_model.get_feature_importance(), 
    index=features, 
    columns=["fi"])
feature_importance = feature_importance.sort_values(by="fi", ascending=False)

print(feature_importance)

                       fi
als_score       30.043771
age             18.356837
average_rating  13.889881
books_read       7.140439
cnt_score        2.887655
reading_years    2.599439
genre_18         2.558679
genre_others     2.522204
genre_1          2.519999
genre_25         2.444803
genre_34         2.211371
books_per_year   1.740982
genre_38         1.588057
genre_20         1.582231
rating_avg       1.570182
genre_33         1.507410
genre_16         1.425483
genre_24         1.334677
genre_5          1.077762
rating_std       0.998138


### Онлайн рекомендации

In [27]:
# получим энкодированные идентификаторы всех объектов, известных нам из events_train
train_item_ids_enc = events_train['item_id_enc'].unique()

max_similar_items = 10

# получаем списки похожих объектов, используя ранее полученную ALS-модель
# метод similar_items возвращает и сам объект, как наиболее похожий
# этот объект мы позже отфильтруем, но сейчас запросим на 1 больше
similar_items = als_model.similar_items(train_item_ids_enc, N=max_similar_items+1)

# преобразуем полученные списки в табличный формат
sim_item_item_ids_enc = similar_items[0]
sim_item_scores = similar_items[1]

similar_items = pd.DataFrame({
    "item_id_enc": train_item_ids_enc,
    "sim_item_id_enc": sim_item_item_ids_enc.tolist(), 
    "score": sim_item_scores.tolist()})
similar_items = similar_items.explode(["sim_item_id_enc", "score"], ignore_index=True)

# приводим типы данных
similar_items["sim_item_id_enc"] = similar_items["sim_item_id_enc"].astype(int)
similar_items["score"] = similar_items["score"].astype("float")

# получаем изначальные идентификаторы
similar_items["item_id_1"] = item_encoder.inverse_transform(similar_items["item_id_enc"])
similar_items["item_id_2"] = item_encoder.inverse_transform(similar_items["sim_item_id_enc"])
similar_items = similar_items.drop(columns=["item_id_enc", "sim_item_id_enc"])

# убираем пары с одинаковыми объектами
similar_items = similar_items.query("item_id_1 != item_id_2")

In [28]:
similar_items[similar_items['item_id_1'] == 7126]

,score,item_id_1,item_id_2
25873,0.948725,7126,7190
25874,0.940997,7126,24280
25875,0.930144,7126,1953
25876,0.925066,7126,58696
25877,0.916340,7126,38296
25878,0.916015,7126,2932
25879,0.913951,7126,7184
25880,0.911433,7126,387749
25881,0.909872,7126,7733
25882,0.909454,7126,30597


In [29]:
similar_items.to_parquet("similar_items.parquet")

In [30]:
def print_sim_items(item_id, similar_items):

    item_columns_to_use = ["item_id", "author", "title", "genre_and_votes", "average_rating", "ratings_count"]
    
    item_id_1 = items.query("item_id == @item_id")[item_columns_to_use]
    display(item_id_1)
    
    si = similar_items.query("item_id_1 == @item_id")
    si = si.merge(items[item_columns_to_use].set_index("item_id"), left_on="item_id_2", right_index=True)
    display(si)

In [ ]:
print_sim_items(7144, similar_items)